In [2]:
import re
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from datasets import load_dataset
from datasets.dataset_dict import DatasetDict, IterableDatasetDict
from datasets.arrow_dataset import Dataset
from datasets.iterable_dataset import IterableDataset
from typing import Union
from tqdm import tqdm

In [3]:
dataset = load_dataset('IlyaGusev/habr', split="train[:1000]")
dataset

Dataset({
    features: ['id', 'language', 'url', 'title', 'text_markdown', 'text_html', 'author', 'original_author', 'original_url', 'lead_html', 'lead_markdown', 'type', 'time_published', 'statistics', 'labels', 'hubs', 'flows', 'tags', 'reading_time', 'format', 'complexity', 'comments'],
    num_rows: 1000
})

In [4]:
def text_cleaner(text: str) -> str:
    """Очистка текста перед разбиением на чанки."""
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)
    return text.strip()


def chunk_documents(documents: Union[DatasetDict, Dataset, IterableDatasetDict, IterableDataset], chunk_size: int = 1000, chunk_overlap: int = 100) -> list[Document]:
    """Разбивает документы на чанки."""
    print(f"Разбиваем {len(documents)} документов на чанки (размер: {chunk_size}, перекрытие: {chunk_overlap})")

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunked_documents: list[Document] = []
    global_chunk_id = 0

    for doc_data in tqdm(documents):
        # Извлекаем текст и метаданные из документа
        text = doc_data.get('text_markdown', '')
        langchain_doc = Document(page_content=text_cleaner(text))
        chunks = text_splitter.split_documents([langchain_doc])

        for j, chunk in enumerate(chunks):
            # Добавляем метаданные в каждый чанк
            chunk.metadata["id"] = doc_data['id']  # ID документа из датасета
            chunk.metadata["author"] = doc_data.get('author', 'Unknown')  # Автор
            chunk.metadata["url"] = doc_data.get('url', '')  # Ссылка на статью
            chunk.metadata["title"] = doc_data.get('title', '')  # Заголовок статьи
            chunk.metadata["document_chunk_id"] = j  # ID чанка внутри документа
            chunk.metadata["global_chunk_id"] = global_chunk_id  # Глобальный ID чанка
            global_chunk_id += 1
            chunked_documents.append(chunk)

    print(f"Создано {len(chunked_documents)} чанков")
    return chunked_documents

In [9]:
chunked_docs = chunk_documents(dataset)
print(f"Length of chunked_docs: {len(chunked_docs)}")
print(f"First chunk: {chunked_docs[0]}")

Разбиваем 1000 документов на чанки (размер: 1000, перекрытие: 100)


100%|██████████| 1000/1000 [00:00<00:00, 1437.54it/s]

Создано 8836 чанков
Length of chunked_docs: 8836
First chunk: page_content='Абитуриенты, поступающие в Университет Чикаго, уже начиная с этой осени, будут обязаны, помимо сдачи основных экзаменов, представить четырехстраничные презентации в формате PowerPoint — на свободную тему.
Частично новые требования связаны с тем, что PowerPoint становится одним из бизнес-инструментов, а кроме того, считают авторы инициативы, в презентации абитуриент сможет раскрыть свои новые качества, которые не смогут проявиться на традиционных экзаменах.
Формат PowerPoint стал своего рода языком делового общения. Ежедневно, по оценкам Microsoft, в мире показывается порядка 30 млн презентаций.
via AP' metadata={'id': 12730, 'author': 'Юлия Благовещенская (skazala)', 'url': 'https://habr.com/ru/post/12730/', 'title': 'Хочешь в университет — сделай презентацию', 'document_chunk_id': 0, 'global_chunk_id': 0}


In [1]:
from langchain_openai import ChatOpenAI
from langchain_mistralai import ChatMistralAI
from langchain_core.messages import SystemMessage, HumanMessage
from pydantic import BaseModel, Field
import os
import time
from dotenv import load_dotenv
from pathlib import Path
import json

load_dotenv()

True

In [5]:
class QA(BaseModel):
    reasoning: str = Field(description="Твои рассуждения о том, почему выбран именно этот вопрос")
    question: str = Field(description="Текст вопроса")
    answer: str = Field(description="Текст ответа")

llm = ChatOpenAI(
    model=os.getenv("MODEL"),
    api_key=os.getenv("KEY"),
    base_url=os.getenv("URL"),
)
llm_qa = llm.with_structured_output(QA)

second_llm = ChatMistralAI(
    model="mistral-small-2506",
    api_key=os.getenv("MISTRAL_API_KEY"),
    timeout=30,
    max_tokens=4096,
    temperature=0.3,
)
second_llm_qa = second_llm.with_structured_output(QA)

QA_GEN_SYSTEM_PROMPT = """
# Role
Ты — эксперт по созданию обучающих данных для тестирования поисковых систем (RAG). Твоя задача — на основе предоставленного фрагмента текста (чанка) сгенерировать пару: "Вопрос пользователя" и "Идеальный ответ".

# Task
Проанализируй предоставленный текст (Context Chunk). Выдели ключевую информацию, которую пользователь мог бы искать. Сформулируй вопрос так, чтобы для ответа на него требовалась эта информация.

# Rules for the QUESTION (Строгие правила):
1. **Самодостаточность:** Вопрос должен быть понятен БЕЗ контекста. Не используй фразы "в этом тексте", "в данном документе", "описанный выше". Представь, что пользователь ничего не читал, а просто задает вопрос в Google или чат-боту.
   - ПЛОХО: "Какие преимущества описаны?"
   - ХОРОШО: "Какие преимущества дает использование Kubernetes в банковской сфере?"
2. **Конкретика:** Избегай общих вопросов типа "О чем текст?". Вопрос должен касаться конкретной детали, процедуры, условия или определения.
3. **Сложность:** Избегай вопросов, на которые можно ответить "Да" или "Нет".

# Rules for the ANSWER (Строгие правила):
1. Ответ должен быть точным и основан ТОЛЬКО на предоставленном тексте.
2. Ответ должен быть полным предложением, а не обрывком фразы.
3. Не начинай ответ с фразы "В тексте сказано...". Просто дай информацию.

# Generation Strategy (Chain of Thought):
Сначала подумай (Reasoning):
1. Какие уникальные сущности (имена, названия, термины) есть в тексте?
2. Какую проблему решает этот текст?
3. Сформулируй вопрос, который содержит эти сущности.

# Output Format
Верни результат СТРОГО в формате JSON:
{
  "reasoning": "Твои рассуждения о том, почему выбран именно этот вопрос",
  "question": "Текст вопроса",
  "answer": "Текст ответа"
}

Если текст не содержит полезной фактической информации (например, это оглавление или "вода"), верни в JSON поле "question": null.

### Examples:

Context: "Для возврата товара заполните форму А-12 и отправьте на почту support@shop.com в течение 14 дней."

BAD Output:
Q: Как вернуть товар?
A: Форма А-12.

GOOD Output:
Q: Какова процедура оформления возврата товара и в какие сроки это нужно сделать?
A: Чтобы оформить возврат, необходимо заполнить форму А-12 и отправить её на электронный адрес support@shop.com. Это нужно сделать не позднее 14 дней с момента покупки.
"""

In [6]:
def call_llm_qa(query: str) -> QA:
    """Вызывает LLM у одного из провайдеров"""
    try:
        response: QA = llm_qa.invoke([
            SystemMessage(content=QA_GEN_SYSTEM_PROMPT),
            HumanMessage(content=f"Chunk: {query}")
        ])
    except:
        response: QA = second_llm_qa.invoke([
            SystemMessage(content=QA_GEN_SYSTEM_PROMPT),
            HumanMessage(content=f"Chunk: {query}")
        ])
    return response

In [ ]:
qa_dataset = []
for sample in chunked_docs[2000:3000:5]:
    # Создаём словарь для хранения результатов
    temp_qa = {
        "id": sample.metadata.get("id", ""),
        "author": sample.metadata.get("author", ""),
        "url": sample.metadata.get("url", ""),
        "title": sample.metadata.get("title", ""),
        "document_chunk_id": sample.metadata.get("document_chunk_id", ""),
        "global_chunk_id": sample.metadata.get("global_chunk_id", ""),
        "question": None,
        "answer": None
    }

    # Вызываем LLM с предусмотрением того, что она может упасть по лимитам или ограничениям
    try:
        response: QA = call_llm_qa(sample.page_content)
    except:
        try:
            time.sleep(5)
            response: QA = call_llm_qa(sample.page_content)
        except:
            continue
    
    # если вопрос или ответ слишком короткие, то скорее всего в чем-то проблема
    # и лучше пропустить такой пример
    if len(response.question) < 15 or len(response.answer) < 15:
        continue
    
    # Обрабатываем результаты
    print(response.question)
    print(response.answer)
    temp_qa["question"] = response.question
    temp_qa["answer"] = response.answer
    qa_dataset.append(temp_qa)
    
    # Сохраняем результаты в файл на каждой итерации, чтоб не потерять прогресс
    with open("qa_dataset.jsonl", "a", encoding="utf-8") as f:
        f.write(json.dumps(temp_qa, ensure_ascii=False) + "\n")
    
    time.sleep(3)

qa_dataset

Какие преимущества дает внедрение MailEssentials на почтовый шлюз предприятия?
Внедрение MailEssentials обеспечивает быструю установку без лишней настройки, защищает от спама и фишинга на уровне почтового шлюза, позволяет отказаться от дорогостоящего развертывания и поддержки защиты на рабочих станциях, не требуется обучать пользователей борьбе со спамом и регулярно настраивать правила фильтрации, а также предотвращает накопление почтового мусора в дисковой подсистеме сервера электронной почты.


[{'id': 304276,
  'author': 'ETitovich',
  'url': 'https://habr.com/ru/post/304276/',
  'title': 'GFI MailEssentials: почта под защитой',
  'document_chunk_id': 1,
  'global_chunk_id': 2000,
  'question': 'Какие преимущества дает внедрение MailEssentials на почтовый шлюз предприятия?',
  'answer': 'Внедрение MailEssentials обеспечивает быструю установку без лишней настройки, защищает от спама и фишинга на уровне почтового шлюза, позволяет отказаться от дорогостоящего развертывания и поддержки защиты на рабочих станциях, не требуется обучать пользователей борьбе со спамом и регулярно настраивать правила фильтрации, а также предотвращает накопление почтового мусора в дисковой подсистеме сервера электронной почты.'}]